In [11]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [12]:
url= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [13]:
soup = BeautifulSoup(url, 'html.parser')
postalCodeList = []
boroughList = []
neighborhoodList = []

In [14]:
soup.find('table').find_all('tr')

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [15]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [16]:
df = pd.DataFrame({"PostalCode": postalCodeList, "Borough": boroughList,"Neighborhood": neighborhoodList})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [17]:
df_dropna = df[df.Borough != "Not assigned"].reset_index(drop=True)
df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [18]:
df_group = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_group

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [19]:
for index, row in df_group.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
test_df = pd.DataFrame(columns=column_names)
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_group[df_group["PostalCode"]==postcode], ignore_index=True)
    
test_df

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Borough,Latitude,Longitude,Neighborhood,PostalCode
0,Downtown Toronto,NaN,NaN,Central Bay Street,M5G
1,North York,NaN,NaN,Hillcrest Village,M2H
2,East York,NaN,NaN,"Woodbine Gardens, Parkview Hill",M4B
3,Scarborough,NaN,NaN,Scarborough Village,M1J
4,East York,NaN,NaN,Leaside,M4G
5,East Toronto,NaN,NaN,Studio District,M4M
6,Scarborough,NaN,NaN,"Maryvale, Wexford",M1R
7,Etobicoke,NaN,NaN,"Albion Gardens, Beaumond Heights, Humbergate, ...",M9V
8,North York,NaN,NaN,Humber Summit,M9L
9,Downtown Toronto,NaN,NaN,"CN Tower, Bathurst Quay, Island airport, Harbo...",M5V


In [23]:
df_group.shape

(103, 3)

In [24]:
import pandas as pd
table = pd.read_csv("http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
table.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
table.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
table.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
df_with_Coordinates = df_group.merge(table, on="PostalCode", how="left")
df_with_Coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [27]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
df_with_Corrdinate_selected = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    df_with_Corrdinate_selected  = df_with_Corrdinate_selected .append(df_with_Coordinates[df_with_Coordinates["PostalCode"]==postcode], ignore_index=True)
    
df_with_Corrdinate_selected 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


In [29]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_with_Corrdinate_selected ['Borough'].unique()),
        df_with_Corrdinate_selected .shape[0]
    )
)

The dataframe has 6 boroughs and 12 neighborhoods.


In [33]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

#clustering based on corrdination
X = df_with_Corrdinate_selected['Latitude']
Y = df_with_Corrdinate_selected['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
df_with_Corrdinate_selected['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(df_with_Corrdinate_selected['Latitude'], df_with_Corrdinate_selected['Longitude'], df_with_Corrdinate_selected['Borough'], df_with_Corrdinate_selected['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map